## Align cells from immature TLS to the reference trajectory
### In this notebook, we will align NaiveB and GCB cells within immature TLS to the reference trajectory

In [31]:
import pandas as pd
import numpy as np
import os
from scipy.stats import pearsonr
from scipy.stats import spearmanr
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import product
import multiprocessing
import multiprocessing as mp
from multiprocessing import Process
from multiprocessing import Pool
import time
from tqdm import tqdm
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

In [16]:
# Load normalized expression matrix
df_data = pd.read_csv('/data/yuchen_data/Stereo_seq/All_Figures/Tls_classification/1.data/Naive_GC_data.txt', index_col=0, sep='\t')
df_data = df_data.loc[df_data.sum(axis=1)>0, ]
df_data.head()

,ST2740P_BIN.103537,ST2740P_BIN.104067,ST2740P_BIN.107778,ST2740P_BIN.113287,ST2740P_BIN.113559,ST2740P_BIN.113560,ST2740P_BIN.113816,ST2740P_BIN.114089,ST2740P_BIN.115463,ST2740P_BIN.116520,...,ST2976T_BIN.79227,ST2976T_BIN.79747,ST2976T_BIN.79748,ST2976T_BIN.79749,ST2976T_BIN.79750,ST2976T_BIN.79751,ST2976T_BIN.80279,ST2976T_BIN.80280,ST2976T_BIN.80281,ST2976T_BIN.80282
CCL19,2.940705,2.715423,0.000000,1.658093,3.834353,3.045612,0.873258,3.819169,0.0,0.0,...,0.000000,2.967565,1.814405,1.813045,1.938066,0.000000,0.000000,3.099077,2.821247,3.492637
CCL21,2.299038,2.342511,0.000000,0.000000,2.010388,1.665935,1.883852,3.347284,0.0,0.0,...,3.802110,4.466763,3.376079,1.813045,2.556499,3.382360,3.286575,2.450033,4.240709,4.916018
FDCSP,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,2.587489,3.070362,3.216301,2.935765,2.792020,3.202956,3.190279,0.000000,0.000000
CR2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,2.587489,1.814405,2.686319,2.556499,2.023283,2.773929,1.137111,0.000000,0.000000
IGHM,0.000000,1.741110,2.265849,0.000000,2.634219,0.887976,1.883852,0.000000,0.0,0.0,...,2.116168,3.355739,3.182976,2.562474,1.602118,2.792020,2.773929,2.248233,2.185927,2.193677


In [18]:
# load bin and TLS information
# load all tls information and init the alignment matrix
df_bin = pd.read_csv('/data/yuchen_data/Stereo_seq/All_Figures/Tls_classification/1.data/Final.B_T.subAnnotation.txt', index_col=0, sep='\t')
df_bin = df_bin[df_bin['SubAnnotation'].isin(['B_GC', 'B_Naive'])]
tmp = []
for idx, row in df_bin.iterrows():
    sample = row['sample']
    tls_id = row['TLS_label_new'].split('_')[1]
    tmp.append(f'{sample}_{tls_id}')
df_bin['TLS_label_merged'] = tmp
df_bin.head()

,sample,SubAnnotation,TLS_label1,TLS_label_new,TLS_label_merged
ST2740P_BIN.103537,ST2740P,B_Naive,ST2740P_19,TLS_19,ST2740P_19
ST2740P_BIN.104067,ST2740P,B_Naive,ST2740P_19,TLS_19,ST2740P_19
ST2740P_BIN.107778,ST2740P,B_Naive,ST2740P_19,TLS_19,ST2740P_19
ST2740P_BIN.113287,ST2740P,B_GC,ST2740P_64,TLS_64,ST2740P_64
ST2740P_BIN.113559,ST2740P,B_Naive,ST2740P_21,TLS_21,ST2740P_21


In [6]:
# load Mature and Immature bin information
df_mature = pd.read_csv('/data/yuchen_data/Stereo_seq/All_Figures/Tls_classification/1.data/Harmony_Mature.df_coor_k21.txt', index_col=0, sep='\t')
df_immature = pd.read_csv('/data/yuchen_data/Stereo_seq/All_Figures/Tls_classification/1.data/Harmony_Immature.df_coor_k23.txt', index_col=0, sep='\t')
df_mature.head()

,x,y,SubAnnotation,color,sample,sample_color,Pseudo_Time_x,Reference_bin
ST2907B_BIN.201119,525.191650,81.065418,B_Naive,#FFD166,ST2907B,#3C15DE,0.000000,1
ST2907B_BIN.201578,520.774171,72.544776,B_Naive,#FFD166,ST2907B,#3C15DE,0.000556,1
ST2907B_BIN.198991,512.150444,104.794047,B_Naive,#FFD166,ST2907B,#3C15DE,0.000556,1
ST2907B_BIN.181638,497.197910,88.971563,B_Naive,#FFD166,ST2907B,#3C15DE,0.001111,1
ST2907B_BIN.202171,499.241309,95.122041,B_Naive,#FFD166,ST2907B,#3C15DE,0.001111,1


In [7]:
# separate the expression matrix to Mature and Immature
df_data_immature = df_data[df_immature.index]
df_data_mature = df_data[df_mature.index]

In [8]:
# load the pseudotime information of cells within mature TLS
df_time = pd.read_csv('/data/yuchen_data/Stereo_seq/All_Figures/Tls_classification/1.data/NaiveB_GC_trajectory_pseudotime.txt', index_col=0, sep='\t')
df_time.sort_values('x', inplace=True)
df_time.head()

,x
ST2907B_BIN.201119,0.000000
ST2907B_BIN.201578,0.000556
ST2907B_BIN.198991,0.000556
ST2907B_BIN.181638,0.001111
ST2907B_BIN.202171,0.001111


In [11]:
# merge cells to 100 pseudocell based on pseudotime
ddd = np.linspace(0, df_time.shape[0], num=101, endpoint=True,retstep=False, dtype=int)
tmp = []
for idx in range(1, len(ddd)):
    tmp.extend([idx]*(ddd[idx]-ddd[idx-1]))
df_time['bin_split'] = tmp
df_time

,x,bin_split
ST2907B_BIN.201119,0.000000,1
ST2907B_BIN.201578,0.000556,1
ST2907B_BIN.198991,0.000556,1
ST2907B_BIN.181638,0.001111,1
ST2907B_BIN.202171,0.001111,1
...,...,...
ST2907B_BIN.197413,973.133551,100
ST2907T_BIN.127336,974.532992,100
ST2907T_BIN.135283,976.090674,100
ST2907T_BIN.133693,977.487381,100


In [12]:
# merge the expression of cells with in each pseudocell
df_data_mature = df_data_mature.T
df_data_mature = pd.merge(df_data_mature, df_time['bin_split'], left_index=True, right_index=True)
df_data_mature_seg = df_data_mature.groupby('bin_split').mean()
df_data_mature_seg = df_data_mature_seg.T
df_data_mature_seg.head()

bin_split,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
CCL19,0.081342,0.435643,0.223455,0.535634,0.302894,0.771386,0.579290,1.033807,1.049936,1.104066,...,1.376075,1.349857,1.092399,0.885835,0.948227,0.691529,0.672102,0.653453,0.536325,0.562664
CCL21,0.182945,0.263336,0.658368,0.877774,0.706418,0.760777,1.243585,0.661765,1.177968,0.555184,...,0.487709,0.545060,0.755983,0.426634,0.518552,0.601407,0.448905,0.544119,0.352239,0.520657
FDCSP,0.000000,0.165968,0.216553,0.177255,0.000000,0.000000,0.113482,0.000000,0.055055,0.186264,...,3.669801,3.905016,3.717184,3.302090,3.473478,3.265701,3.203448,3.000638,2.526969,2.939642
CR2,0.231962,0.000000,0.188881,0.000000,0.000000,0.092332,0.000000,0.000000,0.130518,0.200410,...,3.251866,3.050081,3.282174,2.845111,2.850014,2.706948,2.619177,2.742855,2.763669,2.237870
IGHM,0.683733,0.515409,0.508693,1.257929,1.139477,0.665376,0.849770,1.134594,0.739789,1.159470,...,2.237709,2.309206,1.870462,2.034565,2.230657,1.591795,1.211278,1.428160,1.742672,1.100215


In [13]:
# calculate pairwise correlation between pseudocell in mature TLS with Immature cells
all_pairs = list(product(list(df_data.columns), list(df_data_mature_seg)))
df_data = pd.merge(df_data, df_data_mature_seg, left_index=True, right_index=True)

def calculate_spearmanr(pair):
    x, y = pair
    df_cur = df_data[[x, y]]
    df_cur = df_cur.loc[df_cur.sum(axis=1)>0, ]
    r, p = spearmanr(df_cur[x], df_cur[y])
    if p < 0.05:
        return r
    else:
        return 0

jobs = 20
with multiprocessing.Pool(processes=jobs) as pool:
    results = list(tqdm(
        pool.imap(calculate_spearmanr, all_pairs), 
        total = len(all_pairs)
    ))

df_rst = pd.DataFrame(index=df_immature.index.union(df_data_mature.index), columns=df_data_mature_seg.columns)
for i in range(len(all_pairs)):
    print(i, end='\r')
    x, y = all_pairs[i]
    v = results[i]
    df_rst.loc[x, y] = v

100%|██████████| 864900/864900 [03:22<00:00, 4260.82it/s]


In [15]:
df_rst.head()

bin_split,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
ST2740P_BIN.103537,0,0.181829,0.259725,0.206507,0.229534,0.208193,0.284558,0.249149,0.296871,0.284465,...,0.248225,0.225296,0.226402,0.209056,0.177313,0.175974,0.186202,0.195281,0.167989,0.206199
ST2740P_BIN.104067,0,0.262132,0.276015,0.205559,0.259481,0.281712,0.267243,0.273747,0.301497,0.318615,...,0.25711,0.261199,0.18654,0.191448,0.194919,0.213376,0.187641,0.180809,0.198211,0.181238
ST2740P_BIN.107778,0.229481,0.269727,0.313467,0.208781,0.248777,0.346282,0.29012,0.357738,0.25711,0.286768,...,0.243015,0.220602,0.204074,0.222178,0.182889,0.176871,0.191651,0.137126,0.156055,0.132202
ST2740P_BIN.113287,0.197228,0.26803,0.261641,0.285632,0.341654,0.257155,0.240535,0.308778,0.306046,0.336885,...,0.267637,0.24942,0.240948,0.25282,0.230123,0.202227,0.203717,0.209775,0.201898,0.208036
ST2740P_BIN.113559,0.146111,0.293384,0.282092,0.232678,0.302136,0.343666,0.270981,0.296996,0.415814,0.321227,...,0.320211,0.324446,0.300776,0.322066,0.29795,0.281981,0.260019,0.268827,0.248238,0.250358


In [69]:
# construct the alignment matrix
# anchoring cells to Mature pseudocells based one the max correlation value
# the non-zero value in the alignment matrix indicate that the cells in the TLS were aligned to the corresponding pseudocell
df_plot = pd.DataFrame(index=df_bin['TLS_label_merged'].unique(), columns=df_rst.columns)
df_plot.fillna(0, inplace=True)

dic_tls_pos = {}
for tls in df_bin['TLS_label_merged'].unique():
    dic_tls_pos[tls] = {}
    df_cur = df_bin[df_bin['TLS_label_merged'] == tls]
    for bin in df_cur.index:
        sim = df_rst.loc[bin].sort_values(ascending=False).values[0]
        align_pos = df_rst.loc[bin].sort_values(ascending=False).index[0]
        if align_pos not in dic_tls_pos[tls]:
            dic_tls_pos[tls][align_pos] = [sim]
        else:
            dic_tls_pos[tls][align_pos].append(sim)
for tls, pos_infor in dic_tls_pos.items():
    for pos, values in pos_infor.items():
        df_plot.loc[tls, pos] = np.mean(values)
df_plot.head()

bin_split,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
ST2740P_19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ST2740P_64,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ST2740P_21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.415814,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ST2740P_60,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ST2740P_22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.263811,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Now we are going to fiter TLS using some criteria

#### 1. We filter TLS with fewer than 3 alignments, because they are lacking the continuous changing process, thus may resulting in the inability to develop.

In [73]:
# count the number of aligned cells in each TLS
dic_tls_bin_num = {}
for tls, pos_infor in dic_tls_pos.items():
    c = 0
    for pos, values in pos_infor.items():
        c += len(values)
    dic_tls_bin_num[tls] = c  


# the value of TLS have less than 3 aligned bins will set to 0 in the alignment matrix
tls_filter_BinNumber = []
for tls, number in dic_tls_bin_num.items():
    if number < 3:
        tls_filter_BinNumber.append(tls)
        df_plot.loc[tls] = [0] * df_plot.shape[1]
df_plot = sort_df(df_plot)

In [74]:
# the number of filterd TLS
len(tls_filter_BinNumber)

57

In [75]:
# after filter these TLS, we filtered out the alignment with correlation values below the 25% percentile
corr_values = df_rst.max(axis=1).values
n = 25
threshold = np.percentile(corr_values, n)
df_plot1 = df_plot.copy()
df_plot1[df_plot1 < threshold] = 0
df_plot1 = sort_df(df_plot1)
df_plot1.to_csv(f'/data/yuchen_data/Stereo_seq/All_Figures/Tls_classification/1.data/Align.rst.mean.filter.percentile.{n}.txt', sep='\t')

#### 2. We then filter TLS with number of immune cells < 15 or number of NaiveB < 5

In [76]:
dic_immunecell = {}
dic_gcnaiveB = {}
immune_cells = set(['B', 'Plasma', 'T', 'Myeloid','pDC'])
all_samples = ['ST2907B','ST2740P','ST2772B','ST2772P','ST2772T','ST2837B','ST2846B','ST2846P','ST2893B','ST2893P','ST2893T','ST2896P','ST2903B','ST2903P','ST2907T','ST2931B','ST2931P','ST2931T','ST2976B','ST2976T']
HCC_meta_path = '/data/yuchen_data/Stereo_seq/All_Figures/Figure4/1.data/HCC_metadata_Final/'
for sample in all_samples:
    df_meta = pd.read_csv(f'{HCC_meta_path}/{sample}_metadata_final.txt', index_col=0, sep='\t')
    df_meta = df_meta[df_meta['TLS_final'] != f'{sample}_NA']
    df_ng = df_meta[df_meta['CellSubType'].isin(['GCB', 'Bnaive'])]
    dic_gcnaiveB.update(Counter(df_ng['TLS_final']))
    df_meta = df_meta[df_meta['TLS'] != f'{sample}_NA']
    df_meta = df_meta[df_meta['SpotLight_Anno'].isin(immune_cells)]
    dic_immunecell.update(Counter(df_meta['TLS_final']))

df_tmp1 = pd.DataFrame.from_dict(dic_immunecell, orient='index', columns=['No.ImmuneCells'])
df_tmp2 = pd.DataFrame.from_dict(dic_gcnaiveB, orient='index', columns=['No.NaiveGC'])
df_tls_infor = pd.merge(df_tmp1, df_tmp2, left_index=True, right_index=True)
df_tls_retain = df_tls_infor[(df_tls_infor['No.ImmuneCells'] > 15) & (df_tls_infor['No.NaiveGC'] > 5)]
df_tls_retain

,No.ImmuneCells,No.NaiveGC
ST2907B_2,367,109
ST2907B_5,45,7
ST2907B_20,527,189
ST2907B_18,22,16
ST2907B_24,118,32
...,...,...
ST2976T_6,37,10
ST2976T_25,175,52
ST2976T_27,21,11
ST2976T_29,52,12


### Final, We utilize numpy.polyfit to perform linear regression analysis of the correlation and reference anchor position. 

In [77]:
# get pre-mfuzz matrix

# filter outlier
def filter_outlier(tls):
    align_pos = np.array([int(x) for x in df_plot1.columns[df_plot1.loc[tls] > 0]])
    q1 = np.quantile(align_pos, 0.25)
    q3 = np.quantile(align_pos, 0.75)
    IQR = q3 - q1
    rst = align_pos[((align_pos>(q1-1.5*IQR)) & (align_pos<(q3+1.5*IQR)))]
    return rst

# perform linear regression analysis of the correlation and reference anchor position
# if the No. reference anchor position < 2, we group the TLS into Conforming.
n_para = 1
dic_tls_para = {}
dic_tls_values = {}
conforming_list = []
for tls in df_plot1.index:
    if tls in df_tls_retain.index:
        align_pos = np.array([int(x) for x in df_plot1.columns[df_plot1.loc[tls] > 0]])
        if len(align_pos) >= 2:
            used_pos = filter_outlier(tls)
            v_cur = df_plot1.loc[tls, used_pos].values
            v_cur = v_cur[v_cur > 0]
            z1 = np.polyfit(range(len(v_cur)), v_cur, n_para)
            model = np.poly1d(z1)
            dic_tls_para[tls] = z1
            dic_tls_values[tls] = model(range(10))
        else:
            conforming_list.append(tls)

df_values = pd.DataFrame.from_dict(dic_tls_values, orient='index', columns=range(10))
df_values.head()

,0,1,2,3,4,5,6,7,8,9
ST2931P_41,0.440694,0.377265,0.313837,0.250408,0.186980,0.123552,0.060123,-0.003305,-0.066733,-0.130162
ST2893B_111,0.405022,0.364401,0.323780,0.283159,0.242538,0.201917,0.161296,0.120675,0.080054,0.039433
ST2893T_20,0.388054,0.386048,0.384043,0.382037,0.380031,0.378026,0.376020,0.374014,0.372009,0.370003
ST2772B_62,0.371703,0.392831,0.413959,0.435087,0.456215,0.477343,0.498471,0.519599,0.540727,0.561855
ST2846B_138,0.433338,0.418306,0.403274,0.388242,0.373210,0.358179,0.343147,0.328115,0.313083,0.298051


In [79]:
# save the linear regression result and Deviating TLS information
df_values.to_csv('/data/yuchen_data/Stereo_seq/All_Figures/Tls_classification/1.data//pre.mfuzz.txt', sep='\t')
with open('/data/yuchen_data/Stereo_seq/All_Figures/Tls_classification/1.data/pre.Deviating.tls.txt', 'w') as fo:
    for i in conforming_list:
        fo.write(f'{i}\n')